In [1]:
# !pip uninstall -y transformers
# !mkdir temp && cp -r /kaggle/input/transformers-main-09082023 temp/transformers && cd temp/transformers && python setup.py develop --no-deps

In [2]:
!pip show transformers

Name: transformers
Version: 4.29.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, tokenizers, tqdm
Required-by: 


In [3]:
import sys
sys.path.insert(0, "/kaggle/working/temp/transformers/src/")

In [4]:
import pandas as pd
import io
import torch
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
import requests
from PIL import Image
import re

import torch
import torchvision
import torch.utils.data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2
import numpy as np

import os
from tqdm import tqdm
import math

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
def display_deplot_output(deplot_output, visualize=True):
    '''
    The raw output of deplot
    TITLE | Rural population (%) long-run with 2050 projections<0x0A>(OWID) in Greece<0x0A>Years | Rural population<0x0A>1940 | 47.38<0x0A>1960 | 43.68<0x0A>1980 | 30.28<0x0A>...
    '''

    # x1,x2 <> y1 y2
    # x1 | y1 <0x0A> x2 | y2 <0x0A>
    # label xxxxyyy xyxyxyxy
    deplot_output = deplot_output.replace("<0x0A>", "\n").replace(" | ", "\t")
#     second_a_index = [m.start() for m in re.finditer('\t', deplot_output)][1]
#     last_newline_index = deplot_output.rfind('\n', 0, second_a_index) 

#     title = deplot_output[:last_newline_index]
#     table = deplot_output#[last_newline_index+1:]
    table = deplot_output
    data = io.StringIO(table)
    df = pd.read_csv(data, sep='\t', names=['x', 'y'])
    if visualize:
        display(df)
    
    return df
    

def deplot(path, model, processor, device, visualize=True):
    image = Image.open(path)
    if visualize:
        display(image)
    inputs = processor(images=image, 
                       font_path="../input/arial-font/arial.ttf",
                       text="Generate underlying data table of the figure below:", 
                       return_tensors="pt", 
                       add_special_tokens=True, 
                       max_patches=2048,
                       is_vqa=False)

    # Move inputs to GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.cuda.amp.autocast():
        predictions = model.generate(**inputs, max_new_tokens=512)
#     predictions = model.generate(**inputs, max_new_tokens=512)
    return processor.decode(predictions[0], skip_special_tokens=True)


def deplot_inference(image_path, visualize):
    deplot_output=deplot(image_path, deplot_model, processor, device, visualize)
    print(deplot_output)
    return display_deplot_output(deplot_output, visualize)

In [6]:
deplot_weights_path = '../input/matcha-base/matcha-base'
model_path = '../input/checkpoints/matcha_xyxy_epoch_3_loss_0.7827.bin'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Pix2StructProcessor.from_pretrained(deplot_weights_path)
deplot_model = Pix2StructForConditionalGeneration.from_pretrained(deplot_weights_path).to(device)
deplot_model.resize_token_embeddings(len(processor.tokenizer))
deplot_model.config.text_config.is_decoder = True
deplot_model.encoder.gradient_checkpointing_enable()
deplot_model.decoder.gradient_checkpointing_enable()
deplot_model.load_state_dict(torch.load(model_path))
# processor.is_vqa = False

<All keys matched successfully>

In [7]:
label_map = {'dot': 0, 'horizontal_bar' : 1, 'vertical_bar': 2, 'line': 3, 'scatter': 4}
label_idx_to_classname = {v: k for k, v in label_map.items()}

In [8]:
classification_model = torchvision.models.resnet50(pretrained=False)

num_features = classification_model.fc.in_features
classification_model.fc = nn.Linear(num_features, 5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classification_model = classification_model.to(device)
classification_model.eval()

state_dict = torch.load("../input/checkpoints/Benetech_ResNet50_Acc_87.2093.pth")
classification_model.load_state_dict(state_dict)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [9]:
# val_transforms = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

val_transforms = A.Compose([
    A.Resize(288,288,p=1),
#     A.Resize(500,300,p=1),
#     A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ToTensorV2(),
])

In [10]:
def classification_inference(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img,(500,300))
#     img = img.astype(np.float32)/255.0
    img = val_transforms(image=img)['image']
    
#     print(img.shape)
    
#     inp = torch.stack([img.permute(1, 2, 0)]).cuda()
    inp = torch.stack([img]).cuda()
    out = classification_model(inp).detach().cpu().numpy()
    return label_idx_to_classname[np.argmax(out)]

In [11]:
IMAGE_FOLDER = "/kaggle/input/benetech-making-graphs-accessible/test/images"

all_ids = []
all_values = []
all_chart_types = []

for image_name in tqdm(os.listdir(IMAGE_FOLDER)):
    if ".jpg" not in image_name:
        continue
    try:
        image_path = os.path.join(IMAGE_FOLDER, image_name)

        graph_type = classification_inference(image_path)
        inference_df = deplot_inference(image_path, False)
        
        x_values = inference_df[inference_df.columns[0]].values
        y_values = inference_df[inference_df.columns[1]].values
        
        # check value type for each graph type
        if graph_type in ["horizontal_bar", "vertical_bar", "line", "dot", "scatter"]:
            # x is categorical, y is numerical
            formatted_y_values = []
            for v in y_values:
                try:
                    float(v)
                    if math.isnan(float(v)):
                        formatted_y_values.append(0)
                    else:
                        formatted_y_values.append(v)
                except:
                    formatted_y_values.append(0)
                    
            y_values = formatted_y_values
        
        if graph_type in ["dot", "scatter"]:
            # for dot graph, x could be categorical as well as numerical, how to handle this?
            # just leave all the value be numerical for now

            formatted_x_values = []
            for v in x_values:
                try:
                    float(v)
                    if math.isnan(float(v)):
                        formatted_x_values.append(0)
                    else:
                        formatted_x_values.append(v)
                except:
                    formatted_x_values.append(0)
                    
            x_values = formatted_x_values
            
        
        length = min(len(x_values), len(y_values))
        x_values = ";".join([str(v).strip() for v in x_values][:length])
        y_values = ";".join([str(v).strip() for v in y_values][:length])
    except Exception as e:
        print("Exception", e)
        graph_type = "line"
        x_values = "0;0"
        y_values = "0;0"
    
    image_id = image_name.split(".")[0]

    all_ids.append(image_id + "_x")
    all_values.append(x_values)
    
    all_ids.append(image_id + "_y")
    all_values.append(y_values)
    
    all_chart_types.extend([graph_type, graph_type])

 20%|██        | 1/5 [00:05<00:21,  5.40s/it]

0 | 0 <0x0A> 6 | 0 <0x0A> 12 | 0 <0x0A> 24 | 0


 40%|████      | 2/5 [00:09<00:14,  4.75s/it]

2-1-fed | 88625 <0x0A> 22-Fed | 150400 <0x0A> 22-Fed | 174200 <0x0A> 24-Fed | 180000 <0x0A> 26-Fed | 137800 <0x0A> 27-Fed | 40 <0x0A> 28-Fed | 40 <0x0A> 23-Fed | 60 <0x0A> 21-Mar | 65 <0x0A> 22-Mar | 101 <0x0A> 23-Mar | 101 <0x0A> 10-Mar | 101


 60%|██████    | 3/5 [00:21<00:15,  7.87s/it]

3.0125 | 11 <0x0A> 3.9864565217398645 | 12 <0x0A> 4.0125 | 12 <0x0A> 4.9864565217398645 | 14 <0x0A> 6.0125 | 12 <0x0A> 6.9864565217398645 | 14 <0x0A> 7.9864565217398645 | 18 <0x0A> 8.986456521739864 | 21 <0x0A> 9.986456521739864 | 22 <0x0A> 10.986456521739864 | 23 <0x0A> 11.986456521739864 | 23 <0x0A> 12.9864565217398645 | 24 <0x0A> 13.9864565217398645 | 25 <0x0A> 14.9864565217398645 | 26 <0x0A> 15.9864565217398645 | 29


 80%|████████  | 4/5 [00:22<00:05,  5.27s/it]

Group 1 | 3.6484848484848485 <0x0A> Group 2 | 8.384848484848485


100%|██████████| 5/5 [00:27<00:00,  5.60s/it]

0.0 | 0.013284015784015784 <0x0A> 0.4 | 0.013284015784015784 <0x0A> 0.8 | 0.013284015784015784 <0x0A> 1.2 | 0.013284015784015784 <0x0A> 1.6 | 0.013284015784015784 <0x0A> 2.0 | 0.013284015784015784 <0x0A> 2.4 | 0.013284015784015784


In [12]:
submission_df = pd.DataFrame({
    "id": all_ids,
    "data_series": all_values,
    "chart_type": all_chart_types
})

submission_df.to_csv("submission.csv", index=False)
submission_df

,id,data_series,chart_type
0,000b92c3b098_x,0;6;12;24,line
1,000b92c3b098_y,0;0;0;0,line
2,01b45b831589_x,2-1-fed;22-Fed;22-Fed;24-Fed;26-Fed;27-Fed;28-...,vertical_bar
3,01b45b831589_y,88625;150400;174200;180000;137800;40;40;60;65;...,vertical_bar
4,00f5404753cf_x,3.0125;3.9864565217398646;4.0125;4.98645652173...,scatter
5,00f5404753cf_y,11;12;12;14;12;14;18;21;22;23;23;24;25;26;29,scatter
6,00dcf883a459_x,Group 1;Group 2,vertical_bar
7,00dcf883a459_y,3.648484848484849;8.384848484848485,vertical_bar
8,007a18eb4e09_x,0.0;0.4;0.8;1.2;1.6;2.0;2.4,line
9,007a18eb4e09_y,0.0132840157840157;0.0132840157840157;0.013284...,line


In [13]:
!rm -rf temp